In [1]:
from urllib.request import urlopen
import json

In [2]:
data = []
url = 'https://s3.amazonaws.com/hr-testcases/845/assets/training.json'
data_raw = urlopen(url)
for line in data_raw:
    data.append(json.loads(line))
data.pop(0)

20219

In [3]:
import pandas as pd

df_train = pd.DataFrame.from_dict(data)
df_train

,topic,question,excerpt
0,electronics,What is the effective differencial effective o...,"I'm trying to work out, in general terms, the ..."
1,electronics,Heat sensor with fan cooling,Can I know which component senses heat or acts...
2,electronics,Outlet Installation--more wires than my new ou...,I am replacing a wall outlet with a Cooper Wir...
3,electronics,Buck Converter Operation Question,"i have been reading about the buck converter, ..."
4,electronics,"Urgent help in area of ASIC design, verificati...",I need help with deciding on a Master's Projec...
...,...,...,...
20214,wordpress,How to set a Custom Post Type as the parent of...,I have a Custom Post Type called Recipe with p...
20215,wordpress,Tracking last login and last visit,I'm using the code below to track when a user ...
20216,wordpress,How to exclude the particular category from th...,"add_action( 'pre_get_posts', 'custom_pre_get_p..."
20217,wordpress,display sub categories assoccited with each po...,i have wordpress blog with many posts. each po...


In [4]:
df_train['text'] = df_train[['question','excerpt']].agg(' '.join, axis=1)
df_train

,topic,question,excerpt,text
0,electronics,What is the effective differencial effective o...,"I'm trying to work out, in general terms, the ...",What is the effective differencial effective o...
1,electronics,Heat sensor with fan cooling,Can I know which component senses heat or acts...,Heat sensor with fan cooling Can I know which ...
2,electronics,Outlet Installation--more wires than my new ou...,I am replacing a wall outlet with a Cooper Wir...,Outlet Installation--more wires than my new ou...
3,electronics,Buck Converter Operation Question,"i have been reading about the buck converter, ...",Buck Converter Operation Question i have been ...
4,electronics,"Urgent help in area of ASIC design, verificati...",I need help with deciding on a Master's Projec...,"Urgent help in area of ASIC design, verificati..."
...,...,...,...,...
20214,wordpress,How to set a Custom Post Type as the parent of...,I have a Custom Post Type called Recipe with p...,How to set a Custom Post Type as the parent of...
20215,wordpress,Tracking last login and last visit,I'm using the code below to track when a user ...,Tracking last login and last visit I'm using t...
20216,wordpress,How to exclude the particular category from th...,"add_action( 'pre_get_posts', 'custom_pre_get_p...",How to exclude the particular category from th...
20217,wordpress,display sub categories assoccited with each po...,i have wordpress blog with many posts. each po...,display sub categories assoccited with each po...


In [5]:
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
X_raw, y_train = df_train['text'], df_train['topic']

In [7]:
X_train = []
stemmer = WordNetLemmatizer()

for i in range(0, len(X_raw)):
    document = re.sub(r'\W', ' ', X_raw[i])
    document = document.lower()
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    X_train.append(document)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([('vect', CountVectorizer(stop_words=stopwords.words('english'))),
                  ('tfidf', TfidfTransformer()),
                  ('classifier', MultinomialNB())])

In [9]:
model = pipeline.fit(X_train, y_train)

In [10]:
data_test = []
with open('testcases\input00.txt','r', encoding="utf8") as f:
    lines = f.readlines() 
    for line in lines:
        data_test.append(json.loads(line))
    data_test.pop(0)

In [11]:
test_output = []
with open('testcases\output00.txt','r', encoding="utf8") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        test_output.append(lines[i].rstrip())

In [12]:
df_test = pd.DataFrame.from_dict(data_test)
df_test['topic'] = test_output

df_test['text'] = df_test[['question','excerpt']].agg(' '.join, axis=1)

X_test_raw, y_test = df_test['text'], df_test['topic']

In [13]:
X_test = []

for i in range(0, len(X_test_raw)):
    document = re.sub(r'\W', ' ', X_test_raw[i])
    document = document.lower()
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    X_test.append(document)

In [14]:
predicted = model.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,predicted)

0.885244811069718